In [93]:
import csv
import json
import pandas as pd
from datetime import datetime
import time

vendorMaster = {}
with open('/Users/patelrudra/Documents/BOOKS/Contacts.csv', mode ='r')as file:
    masterFile = csv.reader(file)    
    for vendor in masterFile:
        vendorMaster[vendor[1]] = vendor[0]
        
vendorItem = {}
with open('/Users/patelrudra/Documents/BOOKS/Item.csv', mode ='r')as file1:
    masterFile = csv.reader(file1)    
    for vendor in masterFile:
        vendorItem[vendor[1]] = vendor[0]

vendorgst = {}
with open('/Users/patelrudra/Documents/BOOKS/gst.csv', mode ='r')as file2:
    masterFile = csv.reader(file2)    
    for vendor in masterFile:
        vendorgst[vendor[0]] = vendor[1]        
        
        
gstConfigurationData = {
    "0" : "1357403000000016427",
    "5" : "1357403000000016433",
    "12" : "1357403000000016439",
    "18" : "1357403000000016445",
    "28" : "1357403000000016451"
}
        
with open('/Users/patelrudra/Documents/Medkart/wms/credit note/Credit Notes-csv/Credit Note 004.csv', mode ='r')as file:

    csvFile = csv.reader(file)
    next(csvFile)
    data_list = []
    null_voucher_dict = []
    data={}
    count=0
    Bill=True
    notItems=[]
    
    for entry in csvFile:
        if entry[2] == '':
            if entry[1] == 'Grand Total':
                continue;
            data1 = {
                'item_id': vendorItem[entry[1]],
                'rate':entry[12],
                "quantity":"1",
                "item_order":len(data_list)+1
             }
            data_list.append(data1)
        else 
            data['line_items'] = data_list
            null_voucher_dict.append(data)
            data_list=[]
            date_object = datetime.strptime(entry[0], '%d/%m/%y')
            formatted_date = date_object.strftime("%Y-%m-%d")
            data = {
                'date': formatted_date,
                'customer_id': vendorMaster[entry[1]],
                "creditnote_number" : entry[2],
#                 "gst_no":entry[5],
                 "is_draft":False,
                "is_inclusive_tax": False,
                "reference_number":entry[3],
                "adjustment": entry[17],
                'reference_invoice_type':vendorgst[entry[1]],
#                 'gst_treatment':"business_none",
                'gst_reason':"sales_return",
                "adjustment_description": "adjustment",
             }
            
    for data in null_voucher_dict:
        json_data = json.dumps(data, indent=4)
        with open("sample.json", "a") as outfile:
            outfile.write(json_data + '\n')  

In [94]:
null_voucher_dict.pop(0)
print(len(null_voucher_dict))
null_voucher_dict

113


[{'date': '2023-04-01',
  'customer_id': '1357403000004193217',
  'creditnote_number': '230041SR00000001',
  'is_draft': False,
  'is_inclusive_tax': False,
  'reference_number': '220041SC00032126',
  'adjustment': '-0.17',
  'reference_invoice_type': 'b2c_others',
  'gst_reason': 'sales_return',
  'adjustment_description': 'adjustment',
  'line_items': [{'item_id': '1357403000004358335',
    'rate': '360.87',
    'quantity': '1',
    'item_order': 1}]},
 {'date': '2023-04-01',
  'customer_id': '1357403000004193217',
  'creditnote_number': '230041SR00000002',
  'is_draft': False,
  'is_inclusive_tax': False,
  'reference_number': '230041SC00000059',
  'adjustment': '0.3',
  'reference_invoice_type': 'b2c_others',
  'gst_reason': 'sales_return',
  'adjustment_description': 'adjustment',
  'line_items': [{'item_id': '1357403000004359532',
    'rate': '168.48',
    'quantity': '1',
    'item_order': 1}]},
 {'date': '2023-04-03',
  'customer_id': '1357403000004193217',
  'creditnote_number

In [95]:
import requests
import json

url = "https://books.zoho.in/api/v3/creditnotes?organization_id=60022053195"
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Zoho-oauthtoken 1000.865139a15843133b2e481b90f8d3dfca.098e9350dc50d02431310a71bc06b683'
}
failedData = []
for data in null_voucher_dict:
    payload = json.dumps(data)


    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code >= 200 and response.status_code < 300:  # Assuming a successful response has a status code of 200
        data = response.json()
        
    else:
        print(f"API request failed with status code {response.status_code}")
        response_content = response.content.decode('utf-8')
        failedData.append(data['creditnote_number'])
        print(response_content)
#     time.sleep(0.2)    

print("__________________________________")
print("Finished")
print("__________________________________")
print("failed records bill number : ")
print(failedData)

__________________________________
Finished
__________________________________
failed records bill number : 
[]
